In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os
import time
import ssl

# # For python2
# from urllib.request import urlretrieve
# import tkinter
# from tkinter import filedialog

# def askDialog():
#     return filedialog.askdirectory()

# def inp(text):
#     return raw_input(text)

# For python3
from urllib.request import urlretrieve
import tkinter, tkinter.constants, tkinter.filedialog

def askDialog():
    return tkinter.filedialog.askdirectory()

def inp(text):
    return input(text)

ssl._create_default_https_context = ssl._create_unverified_context


def videoscrape():
    try:
        driver = webdriver.Chrome('D:\計算機程式\chromedriver.exe')
        driver.maximize_window()
        for i in range(1, searchPage + 1):
            url = "https://www.shutterstock.com/video/search/" + searchTerm + "?page=" + str(i)
            driver.get(url)
            print("Page " + str(i))
            for j in range (0, 50):
                while True:
                    container = driver.find_elements_by_xpath("//div[@data-automation='VideoGrid_video_videoClipPreview_" + str(j) + "']")
                    if len(container) != 0:
                        break
                    if len(driver.find_elements_by_xpath("//div[@data-automation='VideoGrid_video_videoClipPreview_" + str(j + 1) + "']")) == 0 and i == searchPage:
                        driver.close()
                        return
                    time.sleep(10)
                    driver.get(url)
                container[0].click()
                while True:
                    wait = WebDriverWait(driver, 60).until(ec.visibility_of_element_located((By.XPATH, "//video[@data-automation='VideoPlayer_video_video']")))
                    video_url = driver.current_url
                    data = driver.execute_script("return document.documentElement.outerHTML")
                    scraper = BeautifulSoup(data, "lxml")
                    video_container = scraper.find_all("video", {"data-automation":"VideoPlayer_video_video"})
                    if len(video_container) != 0:
                        break
                    time.sleep(10)
                    driver.get(video_url)
                video_array = video_container[0].find_all("source")
                video_src = video_array[1].get("src")
                name = video_src.rsplit("/", 1)[-1]
                try:
                    urlretrieve(video_src, os.path.join(scrape_directory, os.path.basename(video_src)))
                    print("Scraped " + name)
                except Exception as e:
                    print(e)
                driver.get(url)
    except Exception as e:
        print(e)
m=0
def imagescrape():
    try:
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        driver = webdriver.Chrome('D:\計算機程式\chromedriver.exe')
        driver.maximize_window()
        for i in range(944, searchPage + 1):
            url = "https://www.shutterstock.com/search?searchterm=" + searchTerm + "&sort=popular&image_type=" + image_type + "&search_source=base_landing_page&language=en&page=" + str(i)
            driver.get(url)
            data = driver.execute_script("return document.documentElement.outerHTML")
            print("Page " + str(i))
            scraper = BeautifulSoup(data, "lxml")
            img_container = scraper.find_all("img", {"class":"z_h_c z_h_e"})
            for j in range(0, len(img_container)-1):
                img_src = img_container[j].get("src")
                name = img_src.rsplit("/", 1)[-1]
                try:
                    urlretrieve(img_src, os.path.join(scrape_directory, os.path.basename(img_src)))
                    print("Scraped " + name)
                except Exception as e:
                    print(e)
        driver.close()
    except Exception as e:
        print(e)

print("ShutterScrape v1.1")

#scrape_directory = "C:/Users/[username]/[path]"

while True:
    while True:
        print("Please select a directory to save your scraped files.")
        scrape_directory = askDialog()
        if scrape_directory == None or scrape_directory == "":
            print("You must select a directory to save your scraped files.")
            continue
        break
    while True:
        searchMode = inp("Search mode ('v' for video or 'i' for image): ")
        if searchMode != "v" and searchMode != "i":
            print("You must select 'v' for video or 'i' for image.")
            continue
        break
    if searchMode == 'i':
        while True:
            image_type = inp("Select image type ('a' for all or 'p' for photo): ")
            if image_type != "a" and image_type != "p":
                print("You must select 'a' for all or 'p' for photo.")
                continue
            break
        if image_type == 'p':
            image_type = 'photo'
        else:
            image_type = 'all'
    while True:
        searchCount = int(inp("Number of search terms: "))
        if searchCount < 1:
            print("You must have at least one search term.")
            continue
        elif searchCount == 1:
            searchTerm = inp("Search term: ")
        else:
            searchTerm = inp("Search term 1: ")
            for i in range (1, searchCount):
                searchTermPart = inp("Search term " + str(i + 1) + ": ")
                if searchMode == "v":
                    searchTerm += "-" + searchTermPart
                if searchMode == "i":
                    searchTerm += "+" + searchTermPart
        break
    while True:
        searchPage = int(input("Number of pages to scrape: "))
        if searchPage < 1:
            print("You must have scrape at least one page.")
            continue
        break
    if searchMode == "v":
        videoscrape()
    if searchMode == "i":
        imagescrape()
    print("Scraping complete.")
    restartScrape = inp("Keep scraping? ('y' for yes or 'n' for no) ")
    if restartScrape == "n":
        print("Scraping ended.")
        break
        
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib import urlretrieve
import os
import Tkinter
import time

def videoscrape():
    try:
        chromeOptions = webdriver.ChromeOptions()
        prefs = {"download.default_directory" : scrape_directory}
        chromeOptions.add_experimental_option("prefs",prefs)
        driver = webdriver.Chrome(chrome_options=chromeOptions)
        driver.maximize_window()
        container_window_handle = None
        while not container_window_handle:
            container_window_handle = driver.current_window_handle
        for i in range(1, searchPage + 1):
            url = "https://www.gettyimages.com/videos/" + searchTerm + "?page=" + str(i)
            driver.get(url)
            print("Page " + str(i))
            for j in range (0, 100):
                while True:
                    container = driver.find_elements_by_xpath("//article[@gi-asset='" + str(j) + "']")
                    if len(container) != 0:
                        break
                    if len(driver.find_elements_by_xpath("//article[@gi-asset='" + str(j + 1) + "']")) == 0 and i == searchPage:
                        driver.close()
                        return
                    time.sleep(10)
                    driver.get(url)
                    print(str(j))
                section = container[0].find_element_by_xpath(".//section[@class='image-section']")
                link = section.find_element_by_xpath(".//a[@class='search-result-asset-link']")
                video_url = link.get_attribute("href")
                driver.get(video_url)
                while True:
                    wait = WebDriverWait(driver, 30).until(ec.visibility_of_element_located((By.XPATH, "//video[@autoplay='true']")))
                    data = driver.execute_script("return document.documentElement.outerHTML")
                    scraper = BeautifulSoup(data, "lxml")
                    video_container = scraper.find_all("video", {"autoplay":"true"})
                    if len(video_container) != 0:
                        break
                    time.sleep(10)
                    driver.get(video_url)
                video_src = video_container[0].get("src")
                name = video_src.rsplit("/", 1)[-1]
                try:
                    driver.get(video_src + "?p=1")
                    print("Scraped " + name)
                except Exception as e:
                    print(e)
                driver.get(url)
    except Exception as e:
        print(e)

print("GettyScrape v1.1")

scrape_directory = "D:\計算機程式\chromedriver.exe"

while True:
    while True:
        print("Please select a directory to save your scraped files.")
        scrape_directory = filedialog.askdirectory()
        if scrape_directory == None or scrape_directory == "":
            print("You must select a directory to save your scraped files.")
            continue
        break
    while True:
        searchCount = input("Number of search terms: ")
        if searchCount < 1:
            print("You must have at least one search term.")
            continue
        elif searchCount == 1:
            searchTerm = raw_input("Search term: ")
        else:
            searchTerm = raw_input("Search term 1: ")
            for i in range (1, searchCount):
                searchTermPart = raw_input("Search term " + str(i + 1) + ": ")
                searchTerm += "-" + searchTermPart
        break
    while True:
        searchPage = input("Number of pages to scrape: ")
        if searchPage < 1:
            print("You must have scrape at least one page.")
            continue
        break
    videoscrape()
    print("Scraping complete.")
    restartScrape = raw_input("Keep scraping? ('y' for yes or 'n' for no) ")
    if restartScrape == "n":
        print("Scraping ended.")
        break

ShutterScrape v1.1
Please select a directory to save your scraped files.
Search mode ('v' for video or 'i' for image): i
Select image type ('a' for all or 'p' for photo): a
Number of search terms: 1
Search term: chihuahua
Number of pages to scrape: 979
Page 944
Scraped cuauhtemoc-chihuahua-mexico-april-19-260nw-1708333936.jpg
Scraped sick-ill-chihuahua-dog-isolated-260nw-1683108211.jpg
Scraped bavarian-chihuahua-dog-owner-isolated-260nw-1147668944.jpg
Scraped cute-chihuahua-portrait-260nw-763354987.jpg
Scraped longhair-chihuahua-portrait-head-one-260nw-419648656.jpg
Scraped wet-long-haired-chihuahua-dogs-260nw-1583194402.jpg
Scraped dogs-terriers-german-shepards-260nw-1043607628.jpg
Scraped adorable-smooth-coat-chihuahua-eyes-260nw-1020833179.jpg
Scraped set-dog-family-vector-illustration-260nw-399042745.jpg
Scraped chihuahua-dog-sit-car-seat-260nw-1034479807.jpg
Scraped chihuahua-dog-portrait-260nw-1016912338.jpg
Scraped studio-shot-different-chihuahua-puppys-260nw-1366714442.jpg
Scra

Scraped dog-basket-yellow-background-260nw-1072141181.jpg
Page 951
Scraped funny-puppy-chihuahua-poses-on-260nw-73395115.jpg
Scraped newborn-puppy-lies-sleeping-cute-260nw-1048898324.jpg
Scraped chihuahua-smile-happily-face-260nw-1048484293.jpg
Scraped female-doctor-using-stethoscope-on-260nw-1495099859.jpg
Scraped small-dog-bag-260nw-693781570.jpg
Scraped cinnamon-white-chihuahua-sitting-comfortable-260nw-699560377.jpg
Scraped siberian-husky-dog-on-beautiful-260nw-609597767.jpg
Scraped vector-dog-yellow-knitted-hat-260nw-1273365850.jpg
Scraped closeup-longhaired-chihuahua-dog-isolated-260nw-1650559015.jpg
Scraped longhair-chihuahua-small-dog-sitting-260nw-225931324.jpg
Scraped bayahibe-dominican-republic-march-2018-260nw-1049914265.jpg
Scraped veterinarian-chihuahua-dog-bandages-on-260nw-672091312.jpg
Scraped chihuahua-smallest-dogthey-two-types-260nw-1040690503.jpg
Scraped dog-playing-swimming-lake-260nw-1103135192.jpg
Scraped two-funny-little-dogs-warm-260nw-782282059.jpg
Scraped ch

Page 958
Scraped small-cute-sleepy-chihuahua-dog-260nw-1040362855.jpg
Scraped large-family-polycarbonate-luggage-small-260nw-408977437.jpg
Scraped portrait-cute-purebred-puppy-chihuahua-260nw-747089728.jpg
Scraped young-woman-pet-dog-260nw-771510682.jpg
Scraped close-small-dog-260nw-1668054709.jpg
Scraped studio-photography-dog-breed-chihuahua-260nw-207247408.jpg
Scraped portrait-cute-purebred-chihuahua-front-260nw-155555165.jpg
Scraped chihuahua-smiling-enjoying-sun-260nw-1530950711.jpg
Scraped portrait-nice-chihuahua-dog-260nw-295754432.jpg
Scraped dwarf-rabbits-puppy-chihuahua-studio-260nw-780121477.jpg
Scraped chihuahuas-on-bench-260nw-1328109017.jpg
Scraped pet-portrait-dog-domestic-chihuahua-260nw-1636609075.jpg
Scraped silhouette-edge-white-dog-wearing-260nw-756181177.jpg
Scraped small-chihuahua-dog-wreath-on-260nw-1407918161.jpg
Scraped boy-home-pet-garden-260nw-1473934823.jpg
Scraped portrait-chihuahua-name-bunny-260nw-1462844828.jpg
Scraped chihuahua-dog-dress-nature-260nw-38

Scraped chihuahua-cuu-mexico-ca-2018-260nw-1725339490.jpg
Scraped profile-side-photo-cute-lady-260nw-1476968474.jpg
Scraped chihuahua-dog-sit-stay-calm-260nw-1163333980.jpg
Scraped little-purebred-puppy-chihuahua-color-260nw-1705525288.jpg
Scraped woman-260nw-409801723.jpg
Scraped chihuahua-dog-park-funny-looking-260nw-1519013021.jpg
Scraped chiwawa-white-fur-sitting-wooden-260nw-1147267868.jpg
Scraped happy-cute-little-dog-wheelchair-260nw-697096144.jpg
Scraped pretty-woman-little-dog-park-260nw-227926633.jpg
Scraped chihuahua-puppy-isolated-on-white-260nw-322579862.jpg
Scraped church-cerocahui-chihuahua-mexico-december-260nw-607316816.jpg
Scraped funny-chihuahuas-playing-garden-260nw-1013039677.jpg
Scraped punk-style-peruvian-hairless-chihuahua-260nw-1454358227.jpg
Page 966
Scraped pup-sitting-260nw-35116339.jpg
Scraped cute-chihuahua-dog-blur-background-260nw-1103366756.jpg
Scraped yorkie-pile-leaves-260nw-1410157520.jpg
Scraped canine-obedience-training-by-happy-260nw-1516975154.jp

Scraped chihuahua-260nw-299449856.jpg
Scraped cute-chihuahua-dog-taking-bath-260nw-793575784.jpg
Scraped portrait-pretty-small-girl-pet-260nw-1499088968.jpg
Scraped chihuahua-dog-running-on-ground-260nw-377900104.jpg
Scraped chihuahua-dog-breed-silhouette-vector-260nw-296853641.jpg
Scraped woman-walking-dogs-park-260nw-474611290.jpg
Scraped black-dog-on-road-260nw-1334274176.jpg
Scraped vector-pattern-chihuahua-bone-260nw-1449482315.jpg
Scraped brown-chihuahua-dog-posing-new-260nw-497059441.jpg
Scraped adorable-chihuahua-dog-posing-city-260nw-673548355.jpg
Page 973
Scraped group-team-dogs-around-blackboard-260nw-340174316.jpg
Scraped portrait-cute-purebred-chihuahua-front-260nw-231654958.jpg
Scraped chihuahua-park-autumn-260nw-1213551307.jpg
Scraped modern-veterinary-clinic-ultrasound-purebred-260nw-1712245582.jpg
Scraped long-haired-chihuahua-lying-down-260nw-1260255037.jpg
Scraped chihuahua-dog-pleading-owner-hugging-260nw-755453749.jpg
Scraped tiny-chihuahua-dolls-dress-260nw-460599

Scraped chihuahua-flowers-isolated-on-white-260nw-138855938.jpg
Scraping complete.
Keep scraping? ('y' for yes or 'n' for no) n
Scraping ended.


ImportError: cannot import name 'urlretrieve'